In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import random
from matplotlib import pyplot as plt
from numba import jit, njit
from spacetime.ca_simulators.CAs import *
%matplotlib inline

In [4]:
def lookup_dict(rule_number):
    '''
    Returns a dictionary which maps ECA neighborhoods to output values. 
    Uses Wolfram rule number convention.
    '''
    neighborhoods = [(0,0,0), (0,0,1), (0,1,0), (0,1,1), (1,0,0), (1,0,1), (1,1,0), (1,1,1)]
    in_binary = '{:{fill}{align}{width}b}'.format(rule_number, 
                                                  fill='0', 
                                                  align='>', 
                                                  width='8')
    
    return dict(zip(neighborhoods, reversed(map(int,in_binary))))

In [5]:
def spacetime_field(rule_number, initial_condition, time_steps):
    '''
    Return a spacetime field array using the given rule number on the 
    given initial condition for the given number of time steps.
    '''
    lookup = lookup_dict(rule_number)
    
    length = len(initial_condition)
    
    # initialize spacetime field and current configuration
    spacetime_field = [initial_condition]
    current_configuration = initial_condition[:]

    # apply the lookup table to evolve the CA for the given number of time steps
    for t in range(time_steps):
        new_configuration = []
        for i in range(len(current_configuration)):

            neighborhood = (current_configuration[(i-1)], 
                            current_configuration[i], 
                            current_configuration[(i+1)%length])

            new_configuration.append(lookup[neighborhood])

        current_configuration = new_configuration
        spacetime_field.append(new_configuration)
    
    return spacetime_field

In [30]:
@njit
def np_lookup(rule_number):
    neighborhoods = [(0,0,0), (0,0,1), (0,1,0), (0,1,1), (1,0,0), (1,0,1), (1,1,0), (1,1,1)]
    in_binary = '{:{fill}{align}{width}b}'.format(rule_number, 
                                                  fill='0', 
                                                  align='>', 
                                                  width='8')
    table = np.empty((2,2,2), dtype=np.uint8)
    for inp, outp in zip(neighborhoods, reversed(map(int,in_binary))):
        table[inp] = outp
    return table

In [7]:
initial = np.random.choice([0,1], 5000)

In [9]:
%timeit spacetime_field(110, initial, 5000)

1 loop, best of 3: 7.22 s per loop


In [10]:
ca = ECA(110, initial)

In [14]:
%%timeit 
ca = ECA(110, initial)
ca.evolve(5000)

1 loop, best of 3: 524 ms per loop


In [15]:
7.22*1000 / 524

13.778625954198473

In [16]:
table = lookup_table(110, 2, 1)

In [18]:
numba_spacetime(initial, table, 5000)
%timeit numba_spacetime(initial, table, 5000)

10 loops, best of 3: 63.7 ms per loop


In [19]:
7.22*1000 / 63.7

113.34379905808477

In [20]:
524 / 63.7

8.226059654631083

In [24]:
@jit
def numba_field_0(rule_number, initial_condition, time_steps):
    '''
    Return a spacetime field array using the given rule number on the 
    given initial condition for the given number of time steps.
    '''
    lookup = lookup_dict(rule_number)
    
    length = len(initial_condition)
    
    # initialize spacetime field and current configuration
    spacetime_field = [initial_condition]
    current_configuration = initial_condition[:]

    # apply the lookup table to evolve the CA for the given number of time steps
    for t in range(time_steps):
        new_configuration = []
        for i in range(len(current_configuration)):

            neighborhood = (current_configuration[(i-1)], 
                            current_configuration[i], 
                            current_configuration[(i+1)%length])

            new_configuration.append(lookup[neighborhood])

        current_configuration = new_configuration
        spacetime_field.append(new_configuration)
    
    return spacetime_field

In [23]:
%timeit numba_field(110, initial, 5000)

1 loop, best of 3: 8.83 s per loop


In [36]:
@njit
def numba_field(lookup, initial_condition, time_steps):
    '''
    Return a spacetime field array using the given rule number on the 
    given initial condition for the given number of time steps.
    '''
    
    length = len(initial_condition)
    
    # initialize spacetime field and current configuration
    spacetime_field = [initial_condition]
    current_configuration = initial_condition[:]

    # apply the lookup table to evolve the CA for the given number of time steps
    for t in range(time_steps):
        new_configuration = np.empty(length, dtype=initial_condition.dtype)
        for i in range(len(current_configuration)):

            neighborhood = (current_configuration[(i-1)], 
                            current_configuration[i], 
                            current_configuration[(i+1)%length])

            new_configuration[i] = lookup[neighborhood]

        current_configuration = new_configuration
        spacetime_field.append(new_configuration)
    
    return spacetime_field

In [37]:
%timeit numba_field(table, initial, 5000)

LoweringError: Caused By:
Traceback (most recent call last):
  File "/home/adam/anaconda2/lib/python2.7/site-packages/numba/compiler.py", line 238, in run
    stage()
  File "/home/adam/anaconda2/lib/python2.7/site-packages/numba/compiler.py", line 621, in stage_nopython_backend
    self._backend(lowerfn, objectmode=False)
  File "/home/adam/anaconda2/lib/python2.7/site-packages/numba/compiler.py", line 576, in _backend
    lowered = lowerfn()
  File "/home/adam/anaconda2/lib/python2.7/site-packages/numba/compiler.py", line 563, in backend_nopython_mode
    self.flags)
  File "/home/adam/anaconda2/lib/python2.7/site-packages/numba/compiler.py", line 858, in native_lowering_stage
    lower.lower()
  File "/home/adam/anaconda2/lib/python2.7/site-packages/numba/lowering.py", line 135, in lower
    self.lower_normal_function(self.fndesc)
  File "/home/adam/anaconda2/lib/python2.7/site-packages/numba/lowering.py", line 176, in lower_normal_function
    entry_block_tail = self.lower_function